# Lab Assignment 2: Commit Message Rectification for Bug-Fixing Commits

**Course:** CS202 Software Tools and Techniques for CSE  
**Date:** 11th August 2025

## Objective
This lab introduces students to the basics of mining open source software (OSS) repositories. The process involves processing and analyzing commits on the GitHub version control system for popular real world projects. The overall aim is to establish a framework for understanding how developers think of bug fixing commits.

## Repository Selection
Selected Repository: **PDFMathTranslate** - A tool for translating mathematical expressions in PDF documents.

In [ ]:
# Clone the selected repository
!git clone https://github.com/Byaidu/PDFMathTranslate.git

In [ ]:
# Import required libraries
import pandas as pd
import re
import torch
import gc
from pydriller import Repository
from transformers import AutoTokenizer, AutoModelForSeq2SeqLM, RobertaTokenizer, RobertaModel
import torch.nn.functional as F
from groq import Groq

# Set repository path
repo_url = 'PDFMathTranslate'

## Bug-Fixing Commit Identification

We use keyword-based matching to identify potential bug-fixing commits. This approach looks for common bug-related keywords in commit messages.

In [ ]:
# Define bug-related keywords for commit identification
bug_keywords = [
    "fixed", "bug", "fixes", "fix", "crash", "solves",
    "resolves", "issue", "regression", "fall back",
    "assertion", "coverity", "reproducible", "stack-wanted",
    "steps-wanted", "testcase", "failure", "fail", "npe",
    "except", "broken", "differential testing", "error",
    "hang", "test fix", "steps to reproduce", "leak",
    "stack trace", "heap overflow", "freez", "problem",
    "overflow", "avoid", "workaround", "break", "stop"
]

def is_bug_commit_naive(commit):
    """Check if a commit is likely a bug fix based on message keywords."""
    message = commit.msg.lower()
    return any(keyword in message for keyword in bug_keywords)

def is_merge_commit(commit):
    """Check if a commit is a merge commit."""
    return len(commit.parents) > 1

In [ ]:
# Extract potential bug-fixing commits
commit_data = []

for commit in Repository(repo_url).traverse_commits():
    if is_bug_commit_naive(commit):
        diff_content = ''
        for file in commit.modified_files:
            diff_content += file.diff if file.diff else ''
            
        commit_data.append({
            "hash": commit.hash,
            "message": commit.msg,
            "parents": [parent for parent in commit.parents],
            "is_merge": commit.merge,
            "diff": diff_content,
            "files_modified": [mod.filename for mod in commit.modified_files]
        })

# Save to CSV
bug_commits_df = pd.DataFrame(commit_data)
bug_commits_df.to_csv("potential_bug_fix_commits.csv", index=False)
print(f"Found and saved {len(bug_commits_df)} potential bug-fixing commits")

## Diff Extraction and LLM Analysis

Using a pre-trained model to analyze code changes and generate fix type predictions.

In [ ]:
# Load the CommitPredictorT5 model for diff analysis
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
tokenizer = AutoTokenizer.from_pretrained("mamiksik/CommitPredictorT5")
model = AutoModelForSeq2SeqLM.from_pretrained("mamiksik/CommitPredictorT5").to(device)
model.eval()

MAX_INPUT_TOKENS = 512
MAX_OUTPUT_TOKENS = 512

def safe_infer(diff_text):
    """Run model inference safely on diff text."""
    if not diff_text:
        return ""
    
    # Tokenize and truncate input
    inputs = tokenizer(
        diff_text,
        return_tensors='pt',
        max_length=MAX_INPUT_TOKENS,
        truncation=True
    ).to(device)

    with torch.no_grad():
        outputs = model.generate(**inputs, max_length=MAX_OUTPUT_TOKENS)

    return tokenizer.decode(outputs[0], skip_special_tokens=True)

In [ ]:
# Analyze files in bug-fixing commits
def collect_file_level_data(repo_url, limit=1000000):
    """Collect file-level data for bug-fixing commits."""
    file_data = []
    processed = 0
    
    for commit in Repository(repo_url).traverse_commits():
        if processed >= limit:
            break
            
        if is_bug_commit_naive(commit):
            for file in commit.modified_files:
                try:
                    inference = safe_infer(file.diff)

                    file_data.append({
                        "hash": commit.hash,
                        "filename": file.filename,
                        "diff": file.diff or "",
                        "llm_inference": inference,
                        "rectified_msg": ""
                    })

                except Exception as e:
                    print(f"Error processing {file.filename} in {commit.hash}: {e}")
                
                # Memory management
                gc.collect()
                if torch.cuda.is_available():
                    torch.cuda.empty_cache()
            
            processed += 1

    return file_data

# Collect and save file-level data
file_level_data = collect_file_level_data(repo_url)
llm_inference_df = pd.DataFrame(file_level_data)
llm_inference_df.to_csv("llm_inference_results.csv", index=False)
print(f"Processed {len(llm_inference_df)} files with LLM inference")

## Commit Message Rectification

Using Groq API to generate improved commit messages based on code changes.

In [ ]:
# Note: Replace 'your_api_key_here' with your actual Groq API key
# groq_client = Groq(api_key="your_api_key_here")

def safe_truncate(text, max_chars=4000):
    """Safely truncate text to prevent token limit issues."""
    if len(text) > max_chars:
        return text[:max_chars] + "\n...[TRUNCATED]..."
    return text

def generate_rectified_message(original_msg, file_changes):
    """Generate improved commit message using Groq API."""
    # Uncomment and modify when API key is available
    # prompt = (
    #     f"Original commit message:\n{safe_truncate(original_msg, 500)}\n\n" +
    #     f"Changes in files:\n{safe_truncate(file_changes, 3000)}\n" +
    #     "Task: Write a concise, accurate commit message summarizing all changes."
    # )
    
    # For demonstration, return a placeholder
    return f"[Rectified] {original_msg[:50]}..."

In [ ]:
# Generate rectified commit messages
def collect_rectified_commits(repo_url, limit=100):
    """Collect commits with rectified messages."""
    rectified_data = []
    processed = 0

    for commit in Repository(repo_url).traverse_commits():
        if processed >= limit:
            break

        if is_bug_commit_naive(commit):
            file_changes = []

            for file in commit.modified_files:
                try:
                    llm_tag = safe_infer(file.diff or "")
                    file_changes.append(f"{file.filename}: {llm_tag}")
                except Exception as e:
                    print(f"Error processing {file.filename}: {e}")
                
                gc.collect()
                if torch.cuda.is_available():
                    torch.cuda.empty_cache()

            # Generate rectified message
            file_changes_text = "\n".join(file_changes)
            rectified_msg = generate_rectified_message(commit.msg, file_changes_text)

            rectified_data.append({
                "hash": commit.hash,
                "developer_msg": commit.msg,
                "rectified_commit_msg": rectified_msg
            })
            
            processed += 1

    return rectified_data

# Collect rectified messages
rectified_commits = collect_rectified_commits(repo_url, limit=50)
rectified_df = pd.DataFrame(rectified_commits)
rectified_df.to_csv("rectified_messages.csv", index=False)
print(f"Generated {len(rectified_df)} rectified commit messages")

## Data Integration and Master Dataset Creation

Combining all collected data into a comprehensive dataset.

In [ ]:
# Load all CSV files
rectified_df = pd.read_csv("rectified_messages.csv")
llm_inference_df = pd.read_csv("llm_inference_results.csv")
bug_commits_df = pd.read_csv("potential_bug_fix_commits.csv")

print("Dataset shapes:")
print(f"Rectified messages: {rectified_df.shape}")
print(f"LLM inference: {llm_inference_df.shape}")
print(f"Bug commits: {bug_commits_df.shape}")

In [ ]:
# Aggregate file-level data by commit hash
aggregated_files = llm_inference_df.groupby("hash").agg({
    "filename": lambda x: " | ".join(x.astype(str)),
    "llm_inference": lambda x: " | ".join(x.astype(str)),
    "diff": lambda x: " | ".join(x.astype(str)),
    "rectified_msg": lambda x: " | ".join(x.dropna().astype(str))
}).reset_index()

# Create master dataset by merging all dataframes
master_df = (
    rectified_df
    .merge(aggregated_files, on="hash", how="left")
    .merge(bug_commits_df, on="hash", how="left")
)

# Clean up columns
master_df = master_df.dropna(axis=1, how='all')

# Save master dataset
master_df.to_csv("master_commits_dataset.csv", index=False)
print(f"Master dataset created with {len(master_df)} entries")
print(f"Columns: {list(master_df.columns)}")

## Evaluation and Analysis

Evaluating the quality of commit message predictions using semantic similarity.

In [ ]:
# Load CodeBERT for semantic similarity evaluation
codebert_tokenizer = RobertaTokenizer.from_pretrained("microsoft/codebert-base")
codebert_model = RobertaModel.from_pretrained("microsoft/codebert-base")

def get_embedding(text):
    """Get CodeBERT embedding for text."""
    tokens = codebert_tokenizer(text, return_tensors='pt', truncation=True, max_length=256)
    with torch.no_grad():
        output = codebert_model(**tokens)
    return output.last_hidden_state[:,0,:]

def calculate_similarity(text1, text2):
    """Calculate cosine similarity between two texts."""
    emb1 = get_embedding(text1)
    emb2 = get_embedding(text2)
    return F.cosine_similarity(emb1, emb2).item()

def normalize_text(val):
    """Normalize text values for comparison."""
    if isinstance(val, list):
        return " ".join(str(x) for x in val)
    elif isinstance(val, str):
        return val
    else:
        return ""

In [ ]:
# Evaluate developer messages vs diff content
def evaluate_messages(df, msg_column, reference_column, threshold=0.95):
    """Evaluate message quality using semantic similarity."""
    total = 0
    hits = 0

    for _, row in df.iterrows():
        msg_text = normalize_text(row[msg_column])
        ref_text = normalize_text(row[reference_column])

        if not msg_text or not ref_text:
            continue

        similarity = calculate_similarity(msg_text, ref_text)
        if similarity > threshold:
            hits += 1
        total += 1

    accuracy = (hits / total * 100) if total > 0 else 0
    return total, hits, accuracy

# Load master dataset for evaluation
master_df = pd.read_csv("master_commits_dataset.csv")

# Evaluate different approaches
print("=== Evaluation Results ===")

# Developer messages vs diff content
total, hits, accuracy = evaluate_messages(master_df, "developer_msg", "diff")
print(f"Developer Messages: {hits}/{total} = {accuracy:.2f}% accuracy")

# LLM inference vs diff content
total, hits, accuracy = evaluate_messages(master_df, "llm_inference", "diff")
print(f"LLM Inference: {hits}/{total} = {accuracy:.2f}% accuracy")

# Rectified messages vs diff content
total, hits, accuracy = evaluate_messages(master_df, "rectified_commit_msg", "diff")
print(f"Rectified Messages: {hits}/{total} = {accuracy:.2f}% accuracy")

## Results Summary

This notebook demonstrates a complete pipeline for:

1. **Repository Mining**: Extracting bug-fixing commits from OSS repositories
2. **Automated Analysis**: Using pre-trained models to analyze code changes
3. **Message Rectification**: Generating improved commit messages
4. **Evaluation**: Measuring the quality of different approaches

### Key Datasets Generated:
- `potential_bug_fix_commits.csv`: Bug-fixing commits identified
- `llm_inference_results.csv`: File-level analysis results
- `rectified_messages.csv`: Improved commit messages
- `master_commits_dataset.csv`: Comprehensive combined dataset

### Applications:
- Dataset creation for automated program repair models
- Multi-versioned program analysis
- Patch generation research
- Software maintainability improvement